In this assignment, you will be making a simple satisficing Minecraft crafting planner.  Given an initial state, you will want to find a series of actions that gets you to a desired goal state.  For fun, you will also use your implementations for Dijkstras and A* to show how they don't do well in a search space this large (without a lot of tweaking).

In this assignment you will:

* Parse and process a data file into a format that is usable in your code
* Create a representation of a state space such that you can check preconditions and apply effects in an efficient manner
* Create a method to convert the intuitive predicate based state system into a propositional system
* Implement Width-Search utilizing Iterative Widening to efficiently find a plan that gets from an arbitrary state space to a desired state space
* Compare Width-Search to optimal search methods such as Dijkstra's and A* to see how they compare in what they can efficiently solve

The goal of this assignment is for you to understand:
* How to manipulate data (a boring, but CRUCIAL skill)
* How to navigate a state space that doesn't map to standard geometry (i.e., domains outside of palt
* How the pruning of a search space can result in a drastic reduction in the time spent performing the search

First we will load in crafting.json,  a json file that

In [1]:
import json
with open('Crafting.json') as f:
    Crafting = json.load(f)

# List of items that can be in your inventory:
print(Crafting['Items'])
# example: ['bench', 'cart', ..., 'wood', 'wooden_axe', 'wooden_pickaxe']

# List of items needed to be in your inventory at the end of the plan:
# (okay to have more than this; some might be satisfied by initial inventory)
print(Crafting['Goal'])
# {'stone_pickaxe': 2}

# Dict of crafting recipes (each is a dict):
print(Crafting['Recipes']['craft stone_pickaxe at bench'])
# example:
# {	'Produces': {'stone_pickaxe': 1},
#	'Requires': {'bench': True},
#	'Consumes': {'cobble': 3, 'stick': 2},
#	'Time': 1
# }


['bench', 'cart', 'coal', 'cobble', 'furnace', 'ingot', 'iron_axe', 'iron_pickaxe', 'ore', 'plank', 'rail', 'stick', 'stone_axe', 'stone_pickaxe', 'wood', 'wooden_axe', 'wooden_pickaxe']
{'stone_pickaxe': 1}
{'Produces': {'stone_pickaxe': 1}, 'Requires': {'bench': True}, 'Consumes': {'cobble': 3, 'stick': 2}, 'Time': 1}


Now, we need to turn these into something that will actually do things.  For each recipe, we will want to make three functions:
    
make_effector, make_checker, make_propositions

make_effector takes in a recipe, and produces a function that takes in a dictionary (representing the state) and returns a new dictionary with the resultant changes of the state (i.e., anything that recipe consumes is consumed and anything that is produced is added)

make_checker takes in a recipe, and produces a function that takes in a  dictionary (representing the state) and returns true if all of the prerequisites of the recipe are found in the state (i.e., anything that recipe consumes is found in sufficient quantity and any equipment required is found)

make_propositions is different -- it takes in a recipe and produces a list of propositions we might expect to see based on that recipe, as well as a function that tells whether that proposition  is true given the state  -- this might not make a ton of sense, so here's an example, 

Consider the recipe: 'craft stone_pickaxe at bench'

This recipe looks like:

`{'Produces': {'stone_pickaxe': 1}, 'Requires': {'bench': True}, 'Consumes': {'cobble': 3, 'stick': 2}, 'Time': 1}`

The propositions involved in this state are:


stone_pickaxe=1

bench=1

cobble=3

stick=2



Note, it's probably for the best to add propositions for all of the intermediary states, i.e.


cobble=2

cobble=1

stick=1


So the result of make_propositions on this recipe would be

`[('stone_pickaxe=1', lambda state: state.get('stone_pickaxe',0) == 1),
('bench=1', lambda state: state.get('bench',0) == 1),
('cobble=3', lambda state: state.get('cobble',0) == 3),
('cobble=2', lambda state: state.get('cobble',0) == 2),
('cobble=1', lambda state: state.get('cobble',0) ==1),
('stick=1', lambda state: state.get('stick',0) ==1),]
`


In [2]:


def make_effector(rule): 
    # do something with rule['Consumes'] and rule['Produces']
    def effect(state):
        #make the function here   
        return state
    return effect

def make_checker(rule):
    # do something with rule['Consumes'] and rule['Requires']
    def check(state):
        #make the function here  
        return False 
    return check

    
def make_propositions(rule):
    # do something with rule['Consumes'], rule['Requires'], and rule['Produces']
    
    return []

Now we will use a named tuple to make a Recipe named tuple that we will use to hold all of the information found in the named tuple.  We will make two lists, 

1)  A list of all of the recipes


2)  A list of all of the propositoins we might find in our state

In [3]:
from collections import namedtuple

Recipe = namedtuple('Recipe',['name','check','effect','cost'])
all_recipes = []
propositions = []
for name, rule in Crafting['Recipes'].items():
    checker = make_checker(rule)
    effector = make_effector(rule)
    recipe = Recipe(name, checker, effector, rule['Time'])
    all_recipes.append(recipe)
    propositions += make_propositions(rule)
    
all_recipes = sorted(all_recipes,key=lambda recipe:recipe.cost)

Provided is a function that will turn a state dictionary into a set of propositions (assuming you've done the work so far)

In [4]:
def propositionalize(state):
    propositionalized_state = set()
    for prop in propositions:
        if prop[1](state):
            propositionalized_state.add(prop[0])
    return propositionalized_state

my_state = {'wood': 0, 'plank': 2, 'stick': 4, 'bench': 1}

print(propositionalize(my_state))

set()


Now comes the iterative widening portion

Given the set of propositions that are important in our state, we want to create a list of all the propositions 
(and combinations up to our max size) that can be found.  At each step of the search we will find all of the unique 
combinations that a state produces (if it does) and return the minimum number of propositions required for uniqueness

e.g., 

if it was the first state with 'bench' it would have width 1

if it was the first state with 'bench' and 'wooden_axe' but nothing of width 1, it would have width 2

if nothing unique can be found it has infinite width


In [5]:
import itertools
proposition_names = set([prop[0] for prop in propositions])

proposition_combos = [ set([prop]) for prop in proposition_names]
MAX_COMBO  = 3
for ii in range(2,MAX_COMBO+1):
    proposition_combos += [ set(prop) for prop in itertools.combinations(proposition_names, ii)]


Now it is time for the width_search.

For each instance of the search, you will want to keep track of a set of all of the seen proposition_combos.  
You will then add to this with a function

`def calculate_width(state)`

that will take in a state, search through all possible proposition_combos, and find whether a combination has been attempted before, and if it hasn't add it to the set.  After going through all combinations, calculate_width should return the minimum width found (and should return float('inf') it can't find a unique state)

The IW search uses breadth-first search, with the key difference being that if the found width of a successor state is greater than MAX_COMBO (3 should suffic here) then that successor state is pruned

Your search should return the sequence of actions required to get you from the initial state to the goal condition

In [6]:
def width_search(initial_state,is_goal):
    def calculate_width(state):
        min_width =  float('inf')
        return  min_width
    
    return []
    
# Try the harder ones once you have ths down

print(width_search({},lambda state: state.get('bench',0) > 0))
#print(width_search({},lambda state: state.get('iron_pickaxe',0) > 0))
#print(width_search({},lambda state: state.get('cart',0) > 0))
        

[]


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\ProgramData\Anaconda3\lib\asyncio\selector_events.py", line 141, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\ProgramData\Anaconda3\lib\asyncio\selector_events.py", line 141, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callb

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\ProgramData\Anaconda3\lib\asyncio\selector_events.py", line 141, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\ProgramData\Anaconda3\lib\asyncio\selector_events.py", line 141, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callb

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\ProgramData\Anaconda3\lib\asyncio\selector_events.py", line 141, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\ProgramData\Anaconda3\lib\asyncio\selector_events.py", line 141, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callb

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\ProgramData\Anaconda3\lib\asyncio\selector_events.py", line 141, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\ProgramData\Anaconda3\lib\asyncio\selector_events.py", line 141, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callb

...

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\ProgramData\Anaconda3\lib\asyncio\selector_events.py", line 141, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\ProgramData\Anaconda3\lib\asyncio\selector_events.py", line 141, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callb

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\ProgramData\Anaconda3\lib\asyncio\selector_events.py", line 141, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\ProgramData\Anaconda3\lib\asyncio\selector_events.py", line 141, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callb

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\ProgramData\Anaconda3\lib\asyncio\selector_events.py", line 141, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\ProgramData\Anaconda3\lib\asyncio\selector_events.py", line 141, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callb

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\ProgramData\Anaconda3\lib\asyncio\selector_events.py", line 141, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\ProgramData\Anaconda3\lib\asyncio\selector_events.py", line 141, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callb

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\ProgramData\Anaconda3\lib\asyncio\selector_events.py", line 141, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\ProgramData\Anaconda3\lib\asyncio\selector_events.py", line 141, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host


Now try out your Dijkstra's and A* implementations (you will have to come up with a heuristic for A*, maybe something like the 10 * number of equipment pieces missing + 5 * number of resources missing -- note this is not admissable, but that's ok, we aren't going for optimality here)